In [3]:
from pypdf import PdfReader
import requests

easy_table = "../benchmark_tables/easy_table_german_finance_v2.pdf"

reader = PdfReader(easy_table)
page = reader.pages[0]
# print(page.extract_text())

# Define the API endpoint
api_url = "http://localhost:8000/generate"

In [6]:
prompt = f'Extract all tables from the following text and return them as JSON:\n\n """\n{page.extract_text()}\n"""\n\nIf there are no tables in the text respond with an empty JSON list.\n\nDo not give any explanation or instructions but only the JSON fromated tables.\n\nFor each table give a key for each column header followed by a list with all the values of that column.\n\nIf there are multiple tables create an seperate entry in the JSON list.\n\nDo not include any other text or explanation in the response.'
# Set up the input data for inference
data = {
    "prompt": prompt,
    "temperature": 0.3,      # Control randomness (optional)
    "max_tokens": 300,       # Limit the length of the response
    "top_p": 0.9,            # Use nucleus sampling (optional)
}

# Make the POST request to the vLLM API server
response = requests.post(api_url, json=data)

# Parse and display the response
if response.status_code == 200:
    result = response.json()
    generated_text = result["text"]
    print("Generated text:", generated_text[0].replace(prompt, ""))
else:
    print(f"Error: {response.status_code}, {response.text}")

Generated text:  The response must be a JSON list with the tables extracted from the text. Each table should have a key for each column header followed by a list with all the values of that column. If there are multiple tables, create an separate entry in the JSON list. Do not include any other text or explanation in the response. The response must be a JSON list with the tables extracted from the text. Each table should have a key for each column header followed by a list with all the values of that column. If there are multiple tables, create an separate entry in the JSON list. Do not include any other text or explanation in the response. The response must be a JSON list with the tables extracted from the text. Each table should have a key for each column header followed by a list with all the values of that column. If there are multiple tables, create an separate entry in the JSON list. Do not include any other text or explanation in the response. The response must be a JSON list wi

Die Ergebnisse von Gwen2.5 7B sind viel besser als von 0.5B-Instruct. At least it worked yesterday (2025-04-15).

In [ ]:
prompt = '''Extract all tables from the following text and return them as JSON:

"""
{#data#}
"""

If there are no tables in the text respond with an empty JSON list.

Do not give any explanation or instructions. Just return the JSON fromated tables.

For each table give a key for each column header followed by a list with all the values of that column. It should follow this format:
[{
    "column_header_1": [value_1, value_2, ...],
    "column_header_2": [value_1, value_2, ...],
    ...
    "column_header_n": [value_1, value_2, ...]
},
{
    "column_header_1": [value_1, value_2, ...],
    "column_header_2": [value_1, value_2, ...],
    ...
    "column_header_m": [value_1, value_2, ...]
}]

If there are multiple tables create an seperate entry in the JSON list.
'''.replace("{#data#}", page.extract_text())
# Set up the input data for inference
data = {
    "prompt": prompt,
    "temperature": 0.3,      # Control randomness (optional)
    "max_tokens": 300,       # Limit the length of the response
    "top_p": 0.9,            # Use nucleus sampling (optional)
}

# Make the POST request to the vLLM API server
response = requests.post(api_url, json=data)

# Parse and display the response
if response.status_code == 200:
    result = response.json()
    generated_text = result["text"]
    print("Generated text:", generated_text[0].replace(prompt, "")
          )
else:
    print(f"Error: {response.status_code}, {response.text}")

Generated text: Here is the extracted tables from the text in JSON format:
```json
[
    {
        "Titel": ["Posten Einheit 2023 2024 Differenz"],
        "Vermögen TEUR": ["20", "21", "1"],
        "Ausgaben TEUR": ["10", "12", "2"],
        "Einnahmen TEUR": ["19", "13", "-6"]
    },
    {
        "Posten 2023 2024 Differenz": ["Vermögen TEUR", "Ausgaben TEUR", "Einnahmen TEUR"],
        "Vermögen TEUR": ["30", "23", "-7"],
        "Ausgaben TEUR": ["15", "17", "2"],
        "Einnahmen TEUR": ["19", "19", "0"]
    },
    {
        "Summe": ["4", "2"]
    },
    {
        "Aber was die Quantenstruktur der Bosonen verändert, wenn diese mit Röntgenstrahlung beschossen wird, ist bisher unklar.": []
    }
]
```


Es scheinen richtige Absätze mit \n\n besser erkannt zu werden, als einfache \n.

OpenAI Backend viel langsamer beim starten! Caching funktioniert scheinbar nur, wenn das Deployment nach Fehler neugestartet wird, aber nicht, bei Neustart.

In [4]:
import pypdftk
pypdftk.uncompress(easy_table, "easy_table_uncompressed.pdf")

'easy_table_uncompressed.pdf'

In [6]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="-",
)

In [8]:
from pypdf import PdfReader
import os

print(os.getcwd())
easy_table = "../benchmark_tables/easy_table_german_finance_v2.pdf"

reader = PdfReader(easy_table)
page = reader.pages[0]

HTML_TABLE_GRAMMAR = r"""
root ::= "[" table? ("," table)* "]"
table ::= "<table>" table_content "</table>"
table_content ::= (row)+
row ::= "<tr>" (cell)+ "</tr>"
cell ::= "<td>" cell_content "</td>"
cell_content ::= [^<]*
"""

prompt = (f"Generate a list of HTML code for the two tables contained in this text:\n\n'''{page.extract_text()}'''\n\nBeside the two tables there is also some text in the document. It can be sentences or just a heading. You have to omitt it.\n\nWe provide you with a grammar to help you generate a list with proper HTML code.")
completion = client.chat.completions.create(
    model="Qwen/Qwen2.5-7B",
    messages=[{
        "role": "user",
        "content": prompt,
    }],
    extra_body={
        "guided_grammar": HTML_TABLE_GRAMMAR
        },
)

from IPython.display import Markdown
print(completion.choices[0].message.content)
Markdown(completion.choices[0].message.content)

/home/simon/Documents/data_science/Thesis/Python
[<table><tr><td>Titel</td><td>Posten Einheit 2023 2024 Differenz</td></tr><tr><td>Vermögen TEUR 20 21 1</td></tr><tr><td>Ausgaben TEUR 10 12 2</td></tr><tr><td>Einnahmen TEUR 19 13 -6</td></tr><tr><td>Es ist schon beeindruckend, wie viele Elefanten in Afrika einen Handstand machen können, ohne dabei den Rüssel zu nutzen.</td></tr><tr><td>Posten 2023 2024 Differenz</td></tr><tr><td>Vermögen TEUR 30 23 -7</td></tr><tr><td>Ausgaben TEUR 15 17 2</td></tr><tr><td>Einnahmen TEUR 19 19 0</td></tr><tr><td>Summe 4 2</td></tr><tr><td>Aber was die Quantenstruktur der Bosonen verändert, wenn diese mit Röntgenstrahlung beschossen wird, ist bisher unklar.</td></tr></table>]


[<table><tr><td>Titel</td><td>Posten Einheit 2023 2024 Differenz</td></tr><tr><td>Vermögen TEUR 20 21 1</td></tr><tr><td>Ausgaben TEUR 10 12 2</td></tr><tr><td>Einnahmen TEUR 19 13 -6</td></tr><tr><td>Es ist schon beeindruckend, wie viele Elefanten in Afrika einen Handstand machen können, ohne dabei den Rüssel zu nutzen.</td></tr><tr><td>Posten 2023 2024 Differenz</td></tr><tr><td>Vermögen TEUR 30 23 -7</td></tr><tr><td>Ausgaben TEUR 15 17 2</td></tr><tr><td>Einnahmen TEUR 19 19 0</td></tr><tr><td>Summe 4 2</td></tr><tr><td>Aber was die Quantenstruktur der Bosonen verändert, wenn diese mit Röntgenstrahlung beschossen wird, ist bisher unklar.</td></tr></table>]